## Spotify Python Client Exploration

### Description

In this notebook, we are going to be exploring the Python client for Spotify leveraging Spotify Developer's REST API.

In particular, we are going to authenticate via the `oauth2` endpoint which leverages the `client_id` and `client_secret` provisioned to an end user on Spotify Developer's Dashboard within a new project.

We're also going to leverage a Spotify artist's uniform resource identifier (URI) in order to query the REST API and retrieve all the albums associated with that specific Spotify artist.

After doing so, we will then take a look at the `results` and essentially create some logic to clean the results for analytics purposes.

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from getpass import getpass

### Authentication

In order to authenticate to the Spotify Python client, we'll need to create a `client_id` and also a `client_secret`. We will feed these variables as input to the `SpotifyClientCredentials` constructor in order to successfully authenticate.

In [4]:
# Create the Values for Client ID and Client Secret
client_id = input("Type in your Client ID: ")
client_secret = getpass("Type in your Client Secret: ")

# Set Up Credentials and Instantiate Client
auth_credential_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret)

### Client Instantiation

In order to instantiate the Python client for Spotify, we need to leverage the `spotipy.Spotify()` constructor. This constructor has an input known as `credentials_manager` which is going to be assigned to our `oauth2` based credentials manager which is stored in the variable `auth_credential_manager` .

In [5]:
client = spotipy.Spotify(client_credentials_manager=auth_credential_manager)

### Parsing Spotify Artist URI ID

In order to get the ID required for a proper Spotify URI, we need to do some string preprocessing on the artist link that we get from the Spotify app based on a given artist.

#### Algorithmic Approach

1. Pass in the artist HTML link or URL as input to the `parse_artist_id()` function.
2. We will split the input string based on the `?` which represents the aspect of the URL where a customizable parameter can be used for querying data.
3. We want to split the first result's element at index 0 by a `/` and then extract the final string from that output.
4. We will then create a string concatenation logic or formatted string logic that essentially takes that result and appends it to the following string: `spotify:artist:`

In [6]:
def parse_artist_id(spotify_artist_link: str) -> str:
    # Separate the original string by the ? in the URL
    q_separate = spotify_artist_link.split("?")[0]

    # Split the q_separate by /
    result = q_separate.split("/")[-1]

    return f"spotify:artist:{result}"


In [7]:
# Testing the Function
parse_artist_id("https://open.spotify.com/artist/1JREmglx633MGQB73njWtE?si=8950fd898a2c44c6")

'spotify:artist:1JREmglx633MGQB73njWtE'

### Retrieving Album Results from Spotify Artist Link

In [15]:
def retrieve_album_results(artist_link: str, album_type: str):
    # Get Valid Spotify Artist URI
    artist_uri = parse_artist_id(artist_link)

    # Assign results to artist_albums() method
    results = client.artist_albums(artist_uri, album_type=album_type)

    return results

### Testing the Retrieve Album Results Function

1. Connor Price
2. Taylor Swift




In [9]:
# Connor Price's Spotify Artist Link
cp_sa_link = "https://open.spotify.com/artist/5zixe6AbgXPqt4c1uSl94L?si=2ea716a651834db3"
# Taylor Swift's Spotify Artist Link
ts_sa_link = "https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02?si=20e5be7237374df0"

In [16]:
# Connor Price's Albums as JSON Response from Spotify API
cp_sa_results = retrieve_album_results(cp_sa_link, 'single')

# Taylor Swift's Albums as JSON Resposne from Spotify API
ts_sa_results = retrieve_album_results(ts_sa_link, 'album')

### Result Exploration

1. Understand the JSON structure of the API response.
2. Get a list of keys that are available to us in regards to album metadata.

In [17]:
# How Many Albums Does Connor Price have?
print(f"Connor Price has {len(cp_sa_results['items'])} albums.")

# What data structure is given back as a response?
print(f"Spotify API returns a {type(cp_sa_results)} structure.")

# Find out the unique keys that exist in the dictionary
cp_sa_results.keys()

# What is the structure of the value associated with the 'items' key?
print(f"The items key has a structure of {type(cp_sa_results['items'])}")



Connor Price has 20 albums.
Spotify API returns a <class 'dict'> structure.
The items key has a structure of <class 'list'>


### Extracting Data from `Items` Key in Spotify API Response

In [32]:
# Looking at Connor Price's items list
sample_cp_sa_result = cp_sa_results['items'][1]

print(sample_cp_sa_result['total_tracks'])

# Look at keys in the sample result dictionary
sample_cp_sa_result.keys()

1


dict_keys(['album_group', 'album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [58]:
# See Types of Values for all Keys in the Dictionary
bool_dct_key_types = list(map(lambda k: type(sample_cp_sa_result[k]) == type(''), sample_cp_sa_result.keys()))

keys = list(sample_cp_sa_result.keys())

str_dct_key_types = [keys[i] for i, val in enumerate(bool_dct_key_types) if val == True]

str_dct_key_types


['album_group',
 'album_type',
 'href',
 'id',
 'name',
 'release_date',
 'release_date_precision',
 'type',
 'uri']

In [62]:
from functools import reduce

def value_transform(dct: dict, str_keys: list):
    # Dictionary Containing only the string keys
    sub_dct = {k:[v] for k,v in dct.items() if k in str_keys}
    return sub_dct

def items_transform(artist_items_list: list, str_keys: list):
    # Map Each Internal Dictionary into the Value Transform
    transformed_items_list = list(map(lambda d: value_transform(d, str_keys), artist_items_list))

    # Instantiate a New Dictionary
    result_dict = {k: [] for k in str_keys}

    # Iterate through each str_key and each item
    for item in transformed_items_list:
        for k, v in item.items():
            result_dict[k].append(v)
    
    return {k: reduce(lambda a,b : a+b, v) for k, v in result_dict.items()}






In [64]:
cp_sa_transformed_results = items_transform(cp_sa_results['items'], str_dct_key_types)

In [67]:
len(cp_sa_transformed_results['uri'])

20